# Chapter 20. 속성 디스크립터

In [1]:
# 지난 시간 복습
# chapter.19 프로퍼티 팩토리 구현하기
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [3]:
li = LineItem('example', 10, -1000)

ValueError: value must be > 0

## LineItem 버전 #1 ~ 4

![LineItem MGN](../images/chap20_mgn.png)   

* 공장과 장치 표기법 (Mills & Gizos Notation, MGN)
    * 클래스는 객체를 생산하는 공장
    * LineItem 공장은 세 가지 속성을 가진 객체를 만듦
    * Quantity 공장에서 만든 객체는 LineItem 객체에 저장된 값을 확인
    
* 디스크립터 클래스
    * 디스크립터 프로토콜을 구현하는 클래스
    * e.g. Quantity
* 디스크립터 객체
    * 디스크립터 클래스의 객체
    * 코드 상 `self`
* 관리 대상 클래스
    * 디스크립터 객체를 클래스 속성으로 선언하는 클래스
    * e.g. LineItem
* 관리 대상 객체
    * 관리 대상 클래스의 객체
    * 코드 상 `instance`
* 저장소 속성
    * 관리 대상 객체 안의 관리 대상 속성값을 담을 속성
    * 항상 클래스 속성
    * e.g. LineItem 객체의 weight와 price
* 관리 대상 속성
    * 디스크립터 객체에 의해 관리되는 괸리 대상 클래스 안의 공개 속성
    * 저장소 속성에 저장

In [5]:
# 별도 모듈 정의 가능한 부분
class Quantity:
    __counter = 0 # 디스크립터 객체를 어떤 변수에 바인딩하는지 구분하기 위해서는 이름을 지정해야 함. 자동으로 이름 지정하기 위해 변수 선언
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        # 관리 대상 속성과 비슷한 이름을 사용하기 위해서는 클래스 데커레이터나 메타 클래스 필요
        self.storage_name = '_{}#{}'.format(prefix, index) 
        cls.__counter +=1 # Quantity 디스크립터가 관리하는 모든 클래스에서 저장소명이 겹치지 않음을 보장
        
    def __get__(self, instance, owner):
        if instance is None: # 클래스에서 관리 대상 속성을 가져올 때 (e.g. LineItem.weight)의 처리
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
# 참고: 위의 Quantity와 동일하게 자동 명명하는 프로퍼티 팩토리
def quantity(storage_name):
    try:
        quantity.counter += 1
    except AttributeError:
        quantity.counter = 0
        
    storage_name = '_{}:{}'.format('quantity', quantity.counter)
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)

In [6]:
li = LineItem('example', 10, -1000)

ValueError: value must be > 0

## LineItem 버전 #5

* description에 빈 값 입력 방지     
* (1) 관리 대상 객체의 저장소 속성을 관리, (2) 속성 설정하기 위한 값 검증 기능을 각각 AutoStorage, Validated로 나눔    

![LineItem #5](../images/chap20_lineitem5.png)

In [9]:
# 별도 모듈화 가능
import abc


class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) 
        cls.__counter +=1 
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(slef, instance, value):
        pass
        
class Quantity(Validated):
    
    def validate(self, instance, value):
        if value < 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value
    
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [10]:
li = LineItem('', 10, 1000)

ValueError: value cannot be empty or blank

## 오버라이딩 디스크립터와 논오버라이딩 디스크립터

* 속성 처리의 비대칭성
    * 속성을 읽을 때
        * 객체에 정의된 속성 반환
        * 객체에 없을 경우 클래스 속성 반환
    * 속성 쓸 때
        * 객체 안에 속성 만듦
        * 클래스에 영향 미치지 않음
        
* 오버라이딩 디스크립터
    * `__set__()` 메서드 구현하는 디스크립터
    * 클래스 속성이지만 객체에 속성 할당하려는 시도 가로챔
    * `__get__()`이 없을 수도 있음
* 논오버라이딩 디스크립터
    * `__set__()` 메서드 구현하지 않는 디스크립터
    * 같은 이름의 객체 속성을 설정하면 디스크립터를 가려 디스크립터가 작동하지 않음
    * 메서드에 이에 해당됨
* 어떠한 디스크립터라도 클래스를 이용하면 덮어써짐 (멍키패칭)

In [17]:
# 보조 함수
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '{} object'.format(cls_name(obj))
    
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
# 예제에서 중요한 클래스
class Overriding:
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class OverridingNoGet:
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class NonOverriding:
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

In [21]:
# 오버라이딩 디스크립터
obj = Managed()
obj.over
Managed.over
obj.over = 7
obj.over
obj.__dict__['over'] = 8
print(vars(obj))
obj.over

-> Overriding.__get__(Overriding object, Managed object, <class Managed>)
-> Overriding.__get__(Overriding object, None, <class Managed>)
-> Overriding.__set__(Overriding object, Managed object, 7)
-> Overriding.__get__(Overriding object, Managed object, <class Managed>)
{'over': 8}
-> Overriding.__get__(Overriding object, Managed object, <class Managed>)


In [23]:
# __get__() 없는 오버라이딩 디스크립터
obj.over_no_get
Managed.over_no_get
obj.over_no_get = 7
obj.over_no_get
obj.__dict__['over_no_get'] = 9
print(obj.over_no_get)
obj.over_no_get = 7
print(obj.over_no_get)

-> OverridingNoGet.__set__(OverridingNoGet object, Managed object, 7)
9
-> OverridingNoGet.__set__(OverridingNoGet object, Managed object, 7)
9


In [24]:
# 논오버라이딩 디스크립터
obj = Managed()
obj.non_over
obj.non_over = 7
print(obj.non_over)
Managed.non_over
del obj.non_over
obj.non_over

-> NonOverriding.__get__(NonOverriding object, Managed object, <class Managed>)
7
-> NonOverriding.__get__(NonOverriding object, None, <class Managed>)
-> NonOverriding.__get__(NonOverriding object, Managed object, <class Managed>)


In [25]:
# 덮어쓰기
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
Managed.over, Managed.over_no_get,Managed.non_over

(1, 2, 3)

In [27]:
# 논오버라이딩 디스크립터로서의 메소드
obj = Managed()
print(obj.spam) # 바인딩된 메서드 객체 반환
print(Managed.spam)
obj.spam = 7
print(obj.spam)

<bound method Managed.spam of <__main__.Managed object at 0x7f843403e860>>
<function Managed.spam at 0x7f8434067048>
7


## 디스크립터 사용에 대한 조언
* 코드를 간결하게 작성하기 위해 프로퍼티 사용
* 읽기 전용 디스크립터는 `__set__()` 구현
* 검증 디스크립터는 `__set__()`만 사용
* 캐시는 `__get()__`에서 효율적으로 구현
* 특별 메소드 이외 메소드는 객체 속성에 가려질 수 있음

## 디스크립터의 문서화 문자열과 관리 대상 속성의 삭제
* 디스크립터 클래스의 문서화 문자열 관리 대상 클래스에 있는 모든 디스크립터 객체를 문서화하기 사용
* `__delete__()`를 통해 관리 대상 속성 삭제